In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
%matplotlib inline
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader

In [ ]:
transform = ToTensor()
train_data = datasets.MNIST(root='data',train=True,transform=transform, download = True)
test_data = datasets.MNIST(root='data',train=False,transform=transform, download = True)
loaders = {'train':DataLoader(train_data,batch_size=100,shuffle=True,num_workers=1),
           'test':DataLoader(test_data,batch_size=100,shuffle=True,num_workers=1)}

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 9.91M/9.91M [00:00<00:00, 16.0MB/s]


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 28.9k/28.9k [00:00<00:00, 489kB/s]


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 1.65M/1.65M [00:00<00:00, 4.41MB/s]


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 4.54k/4.54k [00:00<00:00, 2.30MB/s]


Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [ ]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [ ]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [ ]:
loaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x7db55f1af710>,
 'test': <torch.utils.data.dataloader.DataLoader at 0x7db5564fb390>}

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
class CNN(nn.Module):
  def __init__(self):
    super(CNN,self,).__init__()

    self.conv1 = nn.Conv2d(1,10,kernel_size=5)
    self.conv2 = nn.Conv2d(10,20,kernel_size=5)
    self.drop = nn.Dropout2d()
    self.fc1 = nn.Linear(320,50)
    self.fc2 = nn.Linear(50,10)
  def forward(self,x):
    x = F.relu(F.max_pool2d(self.conv1(x),2))
    x = F.relu(F.max_pool2d(self.drop(self.conv2(x),),2))
    x = x.view(-1,320)
    x=F.relu(self.fc1(x))
    x=F.dropout(x,training=self.training)
    x = self.fc2(x)
    return x
model = CNN().to(device)
optimizer = optim.Adam(model.parameters(),lr=0.001)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
model = CNN().to(device)
optimizer = optim.Adam(model.parameters(),lr=0.001)
loss_fn = nn.CrossEntropyLoss()

def train(epoch):
  model.train()
  for batch_idx,(data,target) in enumerate(loaders['train']):
    data,target = data.to(device),target.to(device)
    optimizer.zero_grad()
    output = model(data)
    loss = loss_fn(output,target)
    loss.backward()
    optimizer.step()
    if batch_idx%50==0:
      print(f'Train Epoch: {epoch}[{batch_idx*len(data)}/{len(loaders["train"].dataset)}({10000. * batch_idx / len(loaders["train"]):.0f}% )]\t{loss.item():.6f}')

def test():
  model.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in loaders['test']:
      data,target = data.to(device),target.to(device)
      output = model(data)
      test_loss += loss_fn(output,target).item()
      pred = output.argmax(dim=1,keepdim=True)
      correct += pred.eq(target.view_as(pred)).sum().item()
  test_loss /= len(loaders['test'].dataset)
  print(f'\nTest set: Average loss: {test_loss:.4f},Accuracy: {correct}/{len(loaders["test"])}')

In [ ]:
for epoch in range(1,16):
  train(epoch)
  test()

Train Epoch: 1[0/60000(0% )]	2.301399
Train Epoch: 1[5000/60000(833% )]	1.292812
Train Epoch: 1[10000/60000(1667% )]	0.732748
Train Epoch: 1[15000/60000(2500% )]	0.600291
Train Epoch: 1[20000/60000(3333% )]	0.689058
Train Epoch: 1[25000/60000(4167% )]	0.544481
Train Epoch: 1[30000/60000(5000% )]	0.442512
Train Epoch: 1[35000/60000(5833% )]	0.523824
Train Epoch: 1[40000/60000(6667% )]	0.323587
Train Epoch: 1[45000/60000(7500% )]	0.492010
Train Epoch: 1[50000/60000(8333% )]	0.461568
Train Epoch: 1[55000/60000(9167% )]	0.639459

Test set: Average loss: 0.0014,Accuracy: 9557/100
Train Epoch: 2[0/60000(0% )]	0.221401
Train Epoch: 2[5000/60000(833% )]	0.280973
Train Epoch: 2[10000/60000(1667% )]	0.238054
Train Epoch: 2[15000/60000(2500% )]	0.544172
Train Epoch: 2[20000/60000(3333% )]	0.485671
Train Epoch: 2[25000/60000(4167% )]	0.418158
Train Epoch: 2[30000/60000(5000% )]	0.342174
Train Epoch: 2[35000/60000(5833% )]	0.282059
Train Epoch: 2[40000/60000(6667% )]	0.323709
Train Epoch: 2[45000/6

In [1]:
model.eval()
data,target = test_data[1024]
data = data.unsqueeze(0).to(device)
output = model(data)
prediction = output.argmax(dim=1,keepdim=True).item()
print(f'Prediction:{prediction}')
image = data.squeeze(0).squeeze(0).cpu().numpy()
plt.imshow(image,cmap='gray')
plt.show

NameError: name 'model' is not defined